## 목포 M(악성) B(양성) 질병 분류
- 컬럼명 : 타켓 제외 모른 상태에서 분석
- 데이터 전처리
    * 두 타켓을 맞추면 됨(이진 분류 문제)
    * 0, 1로 변환(sklearn 의 전처리 모듈을 사용)
    * 전처리 및 표준화 모두 sklearn을 이용하는 것을 목표
    * sklearn 의 pipeline과 컬럼 별 처리를 활용함
    * 데이터 로딩과 dtype 설정 정도로만 pandas 활용
- 모델 선택 classifier 기반 모델을 사용하여야 함
- 모델 선택은 automl 기반 솔루션을 활용(1) 혹은 pipeline처리를 수행

In [166]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()
# one hotencoding

# from sklearn.linear_model import 

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [167]:
import pandas as pd

df = pd.read_csv('wdbc.csv', header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       569 non-null    int64  
 1   1       569 non-null    object 
 2   2       569 non-null    float64
 3   3       569 non-null    float64
 4   4       569 non-null    float64
 5   5       569 non-null    float64
 6   6       569 non-null    float64
 7   7       569 non-null    float64
 8   8       569 non-null    float64
 9   9       569 non-null    float64
 10  10      569 non-null    float64
 11  11      569 non-null    float64
 12  12      569 non-null    float64
 13  13      569 non-null    float64
 14  14      569 non-null    float64
 15  15      569 non-null    float64
 16  16      569 non-null    float64
 17  17      569 non-null    float64
 18  18      569 non-null    float64
 19  19      569 non-null    float64
 20  20      569 non-null    float64
 21  21      569 non-null    float64
 22  22

In [168]:
df[0] = df[0].astype('object')
df[1] = df[1].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   0       569 non-null    object  
 1   1       569 non-null    category
 2   2       569 non-null    float64 
 3   3       569 non-null    float64 
 4   4       569 non-null    float64 
 5   5       569 non-null    float64 
 6   6       569 non-null    float64 
 7   7       569 non-null    float64 
 8   8       569 non-null    float64 
 9   9       569 non-null    float64 
 10  10      569 non-null    float64 
 11  11      569 non-null    float64 
 12  12      569 non-null    float64 
 13  13      569 non-null    float64 
 14  14      569 non-null    float64 
 15  15      569 non-null    float64 
 16  16      569 non-null    float64 
 17  17      569 non-null    float64 
 18  18      569 non-null    float64 
 19  19      569 non-null    float64 
 20  20      569 non-null    float64 
 21  21      569 non-

In [197]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(40,40), dpi=250)
# ax = plt.subplot(1,1,1)
# pd.plotting.scatter_matrix(
#     df[2:], 
#     # c=df[1],
#     figsize=(40,40), 
#     marker='o', 
#     hist_kwds={'bins':15}, 
#     s=1, 
#     ax = ax,
#     alpha=.3
    
# )

# 해당 실행 

In [170]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
num_pipeline = Pipeline(
    steps=[
        # ('std', StandardScaler()),
        # ('imputer', SimpleImputer(strategy=''),
        # ('knnimputer', KNNImputer(n_neighbors=3)),
        ('iqr', RobustScaler()),
    ]
)

cat_pipeline = Pipeline(
    steps = [
        # ('OrdinalEncoder', OrdinalEncoder()),
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore')),
    ]
)

preprocessing = ColumnTransformer(
    [
    ("num",
        num_pipeline, 
     make_column_selector(dtype_include=np.number),
    #  list(range()),
     ),
    ('cat',
     cat_pipeline, 
    #  df.columns[1],
     make_column_selector(dtype_include='category')
    )
    ]
)

model = Pipeline(
    [
    ('preprocess', preprocessing),
    ('model', LogisticRegression(max_iter=1000))
    ]
)
model

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('iqr',
                                                                   RobustScaler(copy=True,
                                                                                quantile_range=(25.0,
                                                                                                75.0),
                                                                                unit_variance=False,
                                                                                with_centering=True,
                                                                                with_scaling=True))],
                                                           verbose=False),
                                                  <sklearn.compose._column_transfor...
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017585282D90>)],
                                   verbose=False,
                                   verbose_feature_names_out=True)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [191]:

X = df[df.columns[2:]]
X.columns = range(X.shape[1])
X.reset_index(drop=True, inplace=True)
# result columns
y = df[df.columns[1]]
# y.reset_index(drop=True, inplace=True)


In [192]:
X.shape, y.shape

((569, 30), (569,))

In [193]:


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)



In [194]:
model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           B       0.97      1.00      0.99        72
           M       1.00      0.95      0.98        42

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



In [139]:
from pycaret.classification import *

test  =df[df.columns[1:]]
test[test.columns[0]]

0      M
1      M
2      M
3      M
4      M
      ..
564    M
565    M
566    M
567    M
568    B
Name: 1, Length: 569, dtype: category
Categories (2, object): ['B', 'M']

In [195]:

s = setup(
    data=test,
    target=0,
    normalize=True,
    # normalize_method='robust',
    # use_gpu=True,
    transformation=True,
)

,Description,Value
0,Session id,4254
1,Target,1
2,Target type,Binary
3,Target mapping,"B: 0, M: 1"
4,Original data shape,"(569, 31)"
5,Transformed data shape,"(569, 31)"
6,Transformed train set shape,"(398, 31)"
7,Transformed test set shape,"(171, 31)"
8,Numeric features,30
9,Preprocess,True


c:\Users\Administrator\anaconda3\envs\EXAM_ML\lib\site-packages\pycaret\internal\metrics.py:51: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [196]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9849,0.0000,0.9849,0.9858,0.9848,0.9673,0.9684,0.3350
knn,K Neighbors Classifier,0.9774,0.0000,0.9774,0.9777,0.9773,0.9514,0.9519,0.2740
catboost,CatBoost Classifier,0.9698,0.0000,0.9698,0.9713,0.9697,0.9352,0.9368,1.3550
svm,SVM - Linear Kernel,0.9672,0.0000,0.9672,0.9693,0.9670,0.9291,0.9316,0.0160
ridge,Ridge Classifier,0.9672,0.0000,0.9672,0.9682,0.9669,0.9286,0.9301,0.0160
rf,Random Forest Classifier,0.9648,0.0000,0.9648,0.9663,0.9647,0.9245,0.9261,0.0190
lightgbm,Light Gradient Boosting Machine,0.9648,0.0000,0.9648,0.9662,0.9647,0.9243,0.9259,0.0670
lda,Linear Discriminant Analysis,0.9647,0.0000,0.9647,0.9664,0.9643,0.9228,0.9253,0.0160
xgboost,Extreme Gradient Boosting,0.9647,0.0000,0.9647,0.9674,0.9648,0.9249,0.9274,0.0330
ada,Ada Boost Classifier,0.9624,0.0000,0.9624,0.9640,0.9622,0.9189,0.9209,0.0260
